# AI Meal Coach: Health-Aware AI Meal Planner
* A GenAI-powered agent that helps you plan healthy meals based on your dietary needs and fridge contents.

# AI Meal Coach: Health-Aware AI Meal Planner
*A GenAI-powered agent that helps you plan healthy meals based on your dietary needs and fridge contents.*

---

## 🧠 1. Project Overview

This notebook presents a generative AI agent that recommends recipes tailored to your dietary restrictions and available ingredients.  
Users can input what they have in their fridge, specify a health profile (e.g., Hashimoto, diabetes), and receive meal suggestions that match both.

**Key GenAI capabilities demonstrated:**
- Retrieval-Augmented Generation (RAG)
- Health-aware reasoning with Chain-of-Thought
- AI-based recipe adaptation and explanation




## 📦 2. Ingredients: Dataset and Setup

### 2.1 Import libraries and install dependencies

```python
# TODO: install langchain, faiss, google-generativeai, etc.
# TODO: import pandas, numpy, json, relevant thing from langchain, etc.

In [1]:
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU google-generativeai 
!pip install -qU faiss-cpu fsspec==2024.10.0 gcsfs==2024.10.0
!pip install -qU langchain-google-genai  
!pip install -qU langgraph
!pip install -qU langgraph-prebuilt 
#!pip check # weryfikacja pakietów

# # Remove conflicting packages from the Kaggle base environment.
# !pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
# # Install langgraph and the packages used in this lab.
# !pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 46.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.29.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.16.0, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 18.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [2]:
# necessary imports
import pandas as pd
import google.generativeai as genai
import json
import numpy as np
from tqdm import tqdm
import faiss
from typing import TypedDict, List, Optional
import logging

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/epirecipes/recipe.py
/kaggle/input/epirecipes/utils.py
/kaggle/input/epirecipes/full_format_recipes.json
/kaggle/input/epirecipes/epi_r.csv


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)
print("done")

done


### 2.2 Load recipe datasets

In [5]:
# === EPICURIOUS ===
epicurious_path = "/kaggle/input/epirecipes/full_format_recipes.json"
with open(epicurious_path, "r") as f:
    epicurious_raw = json.load(f)
    
df_epi = pd.DataFrame(epicurious_raw)

In [6]:
print("Epicurious columns:", df_epi.columns.tolist())

Epicurious columns: ['directions', 'fat', 'date', 'categories', 'calories', 'desc', 'protein', 'rating', 'title', 'ingredients', 'sodium']


In [7]:

df_epi["ingredients"] = df_epi["ingredients"].apply(lambda x: ", ".join(x) if isinstance(x, list) else "")
df_epi["text_for_embedding"] = df_epi["title"] + ". Ingredients: " + df_epi["ingredients"]

In [8]:
df_epi_small = df_epi[["title", "text_for_embedding"]].copy()

df_epi_small = df_epi_small[df_epi_small["text_for_embedding"].notna()]
df_epi_small = df_epi_small[df_epi_small["text_for_embedding"].str.strip() != ""]

### 2.3 Define dietary profiles

In [9]:
diet_profiles = [
    {
        "name": "Hashimoto",
        "description": "Autoimmune thyroid disorder requiring reduced iodine and anti-inflammatory foods",
        "restrictions": ["no gluten", "no lactose", "low iodine"],
        "excluded_ingredients": ["soy", "kale", "broccoli", "processed food"],
        "preferences": ["anti-inflammatory", "high fiber"]
    },
    {
        "name": "Type 2 Diabetes",
        "description": "Metabolic disorder that affects blood sugar regulation",
        "restrictions": ["low sugar", "low refined carbohydrates"],
        "excluded_ingredients": ["sugar-sweetened beverages", "white bread", "candy", "baked goods"],
        "preferences": ["low glycemic index", "high fiber", "balanced carbohydrates"]
    },
    {
        "name": "Celiac Disease",
        "description": "Autoimmune disorder triggered by gluten consumption",
        "restrictions": ["no gluten"],
        "excluded_ingredients": ["wheat", "barley", "rye", "malt", "processed gluten-containing foods"],
        "preferences": ["naturally gluten-free whole foods", "high fiber"]
    },
    {
        "name": "Vegan Diet",
        "description": "Plant-based diet excluding all animal products",
        "restrictions": ["no meat", "no dairy", "no eggs", "no animal-derived ingredients"],
        "excluded_ingredients": ["meat", "milk", "cheese", "honey", "gelatin"],
        "preferences": ["whole plant foods", "plant-based proteins", "B12-fortified foods"]
    },
    {
        "name": "High-Protein (Athletes)",
        "description": "Diet supporting muscle growth and recovery with increased protein intake",
        "restrictions": [],
        "excluded_ingredients": ["excess sugar", "deep-fried food"],
        "preferences": ["high protein", "complex carbohydrates", "healthy fats", "lean meats", "legumes", "dairy or alternatives"]
    }
]

## 🔍 3. Recipe Embedding and Semantic Search

### 3.1 Create embeddings for recipes

In [10]:
# embedding function:
def embed_texts_genai(texts: list[str], model="models/embedding-001", task_type="retrieval_document", title=None) -> np.ndarray:
    all_embeddings = []
    for text in tqdm(texts):
        response = genai.embed_content(
            model=model,
            content=text,
            task_type=task_type,
            title=title or "Recipe"
        )
        all_embeddings.append(response["embedding"])
    return np.array(all_embeddings)

# texts = df_epi_small["text_for_embedding"].tolist()

# recipe_embeddings = embed_texts_genai(texts)

# print("✅ Embedding shape:", recipe_embeddings.shape)

### 3.2 Store embeddings in vector database

In [11]:
# recipe_embeddings = recipe_embeddings.astype("float32")

# index = faiss.IndexFlatL2(recipe_embeddings.shape[1])
# index.add(recipe_embeddings)

# print("🔍 FAISS index size:", index.ntotal)

# # save index of database to a file
# faiss.write_index(index, "recipe_index.faiss")

In [12]:
# #save embeddings to a file for further use
# df_epi_small.reset_index(drop=True, inplace=True)  
# df_epi_small.to_csv("recipe_metadata.csv", index=False)

## Użycie wcześniejsszego embeddingu

In [13]:
!pip install gdown

In [14]:
# load previously made embeddings to save time
import gdown
# Google Drive ID 
index_file_id = "1sWrwfcJ6px3n-l2CLIpK4KG7D3GKl-kG"
csv_file_id = "189pU4qLUiB2C_qmx5KEVzoQ1YfZnRBOm"

# local path
index_local_path = "recipe_index.faiss"
csv_local_path = "recipe_metadata.csv"

gdown.download(f"https://drive.google.com/uc?id={index_file_id}", index_local_path, quiet=False)
gdown.download(f"https://drive.google.com/uc?id={csv_file_id}", csv_local_path, quiet=False)

index = faiss.read_index(index_local_path)
metadata = pd.read_csv(csv_local_path)

print("Index i metadane załadowane poprawnie!")

Downloading...
From: https://drive.google.com/uc?id=1sWrwfcJ6px3n-l2CLIpK4KG7D3GKl-kG
To: /kaggle/working/recipe_index.faiss
100%|██████████| 61.8M/61.8M [00:00<00:00, 206MB/s]
Downloading...
From: https://drive.google.com/uc?id=189pU4qLUiB2C_qmx5KEVzoQ1YfZnRBOm
To: /kaggle/working/recipe_metadata.csv
100%|██████████| 8.97M/8.97M [00:00<00:00, 140MB/s]


Index i metadane załadowane poprawnie!


### 3.3 Create search function: input → matching recipes

In [15]:
def search_recipes(query: str, diet_profile_name: str = None, top_k: int = 3):
    """Embeds the user's query and search for top_k compatible recipes, 
    optionally filtered by dietary profile."""
    # print(f"Jestem w search_recipes. query: {str} diet: {diet_profile_name}") #By PIotr
    # selec dietary profile (optional)
    selected_profile = None
    if diet_profile_name:
        selected_profile = next((p for p in diet_profiles if p["name"].lower() == diet_profile_name.lower()), None)
        if selected_profile is None:
            raise ValueError(f"Diet profile '{diet_profile_name}' not found.")
    
    # query embedding
    query_embedding = embed_texts_genai([query])
    query_embedding = query_embedding.astype("float32")
    
    # search top_k recipes
    distances,indices = index.search(query_embedding, top_k * 2)
    
    results = []
    for idx, dist in zip(indices[0], distances[0]):
        title = df_epi_small.iloc[idx]["title"]
        ingredients_text = df_epi_small.iloc[idx]["text_for_embedding"].lower()
        
        # filtering by excluded_ingredients
        if selected_profile:
            if any(ex_ingredient.lower() in ingredients_text for ex_ingredient in selected_profile["excluded_ingredients"]):
                continue
        results.append({
            "title": title,
            "ingredients": ingredients_text,
            "distance": dist
        })

        if len(results) >= top_k:
            break
    return results

## 🧑‍🍳 4. Building the GenAI Agent

### 4.1 Define Tools (functions for search, filtering, planning)

In [16]:
from langchain.agents import Tool

def search_recipes_tool_wrapper(query: str) -> str:
    print("Tool search_recipes_tool_wrapper ")
    """
    Tool wrapper that assumes the query is in the format:
    'ingredients: zucchini, tofu, onion | diet: Hashimoto'
    """
    try:
        #  Parsing components and profile
        if "|" in query:
            ingredients_part, diet_part = query.split("|")
            ingredients = ingredients_part.replace("ingredients:", "").strip()
            diet = diet_part.replace("diet:", "").strip()
        else:
            ingredients = query.strip()
            diet = None

        results = search_recipes(ingredients, diet_profile_name=diet, top_k=3)
        if not results:
            return "No recipes found for this query and dietary profile."
            
        return "\n\n".join([f"{r['title']} (distance: {r['distance']:.2f})\nIngredients: {r['ingredients']}" for r in results])

    except Exception as e:
        return f"Error in recipe search: {str(e)}"

In [17]:
search_tool = Tool(
    name="SearchRecipeByIngredientsAndDiet",
    func=search_recipes_tool_wrapper,
    description="Searches for recipes based on given ingredients and an optional dietary profile. Input format: 'ingredients: zucchini, tofu | diet: Hashimoto'"
)

In [18]:
def adapt_recipe_to_diet(recipe_title: str, ingredients: str, directions: str, target_diet: str) -> str:
    print("Tool adapt_recipe_to_diet ")  # By Piotr
    """
    Uses GenAI to rewrite a recipe so that it fits the selected dietary profile.
    """
    prompt = f"""You are a health-aware recipe assistant.
    Rewrite the following recipe to make it suitable for a {target_diet} diet.
    
    Title: {recipe_title}

    Ingredients:
    {ingredients}

    Directions:
    {directions}

    Return a revised version with adjusted ingredients and cooking steps.
    """

    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(prompt)
    return response.text 


def generate_meal_plan(diet_profile_name: str) -> str:
    print("Tool generate_meal_plan ")  # By PIotr
    """
    Generates a simple 1-day meal plan based on a dietary profile using LLM.
    """
    prompt = f"""Create a one-day healthy meal plan for someone following a {diet_profile_name} diet.
    The plan should include breakfast, lunch, dinner, and snacks, with simple dish names and a short description.

    Format:
    Meal Type: Name - Description
    """
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(prompt)
    return response.text
    

def explain_diet_compatibility(recipe_title: str, ingredients: str, diet_profile_name: str) -> str:
    """
    Uses LLM to reason whether the recipe fits the selected dietary profile and explain why.
    """
    prompt = f"""Analyze the following recipe and determine if it is compatible with a {diet_profile_name} diet.

    Title: {recipe_title}

    Ingredients:
    {ingredients}

    Give a short reasoning about whether this recipe fits the dietary profile or not, and why.
    """
    model = genai.GenerativeModel("gemini-2.0-flash")
    response = model.generate_content(prompt)
    return response.text

In [19]:
adapt_recipe_tool = Tool(
    name="AdaptRecipeToDiet",
    func=lambda query: adapt_recipe_to_diet_wrapper(query),
    description=(
        "Rewrites a recipe to make it compatible with a specific diet. "
        "Input format: 'title: [recipe title] | ingredients: [ingredients] | directions: [directions] | diet: [diet name]'"
    )
)

def adapt_recipe_to_diet_wrapper(query: str) -> str:
    print("Tool adapt_recipe_to_diet_wrapper ")  # By Piotr
    try:
        parts = {k.strip(): v.strip() for k, v in (item.split(":", 1) for item in query.split("|"))}

        title = parts.get("title", "")
        ingredients = parts.get("ingredients", "")
        diet = parts.get("diet", "")
        
        return adapt_recipe_to_diet(
            recipe_title=parts.get("title", ""),
            ingredients=parts.get("ingredients", ""),
            directions=parts.get("directions", ""),
            target_diet=parts.get("diet", "")
        )
    except Exception as e:
        return f"[ERROR] Could not adapt recipe: {str(e)}"

meal_plan_tool = Tool(
    name="GenerateMealPlan",
    func=lambda query: generate_meal_plan_wrapper(query),
    description="Generates a 1-day healthy meal plan for a given dietary profile. Input format: 'diet: [diet name]'"
)
def generate_meal_plan_wrapper(query: str) -> str:
    print("Tool generate_meal_plan_wrapper ")  # By Piotr
    try:
        parts = {k.strip(): v.strip() for k, v in (item.split(":", 1) for item in query.split("|"))}
        return generate_meal_plan(diet_profile_name=parts.get("diet", ""))
    except Exception as e:
        return f"[ERROR] Could not generate meal plan: {str(e)}"

diet_explanation_tool = Tool(
    name="ExplainDietCompatibility",
    func=lambda query: explain_diet_compatibility_wrapper(query),
    description=(
        "Explains if a recipe is compatible with a given diet and why. "
        "Input format: 'title: [recipe title] | ingredients: [ingredients] | diet: [diet name]'"
    )
)

def explain_diet_compatibility_wrapper(query: str) -> str:
    try:
        parts = {k.strip(): v.strip() for k, v in (item.split(":", 1) for item in query.split("|"))}
        return explain_diet_compatibility(
            recipe_title=parts.get("title", ""),
            ingredients=parts.get("ingredients", ""),
            diet_profile_name=parts.get("diet", "")
        )
    except Exception as e:
        return f"[ERROR] Could not explain compatibility: {str(e)}"

In [20]:
# Wrap the tools with debugging
def debug_tool_wrapper(tool_func):
    def wrapped_tool(*args, **kwargs):
        print(f"Debug: Invoking tool {tool_func.__name__} with args={args}, kwargs={kwargs}")
        result = tool_func(*args, **kwargs)
        print(f"Debug: Tool {tool_func.__name__} returned {result}")
        return result
    return wrapped_tool

# Wrap each tool
search_tool.func = debug_tool_wrapper(search_tool.func)
adapt_recipe_tool.func = debug_tool_wrapper(adapt_recipe_tool.func)
meal_plan_tool.func = debug_tool_wrapper(meal_plan_tool.func)
diet_explanation_tool.func = debug_tool_wrapper(diet_explanation_tool.func)

In [21]:
tools = [
    search_tool,                 
    adapt_recipe_tool,
    meal_plan_tool,
    diet_explanation_tool
]

### 4.2 Prompt templates and agent logic

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY, temperature=0.3)

In [23]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

conversation_config = {
    "configurable": {
        "thread_id": "user"  
    }
}

In [24]:
system_prompt = (
    "You are a helpful, health-conscious AI cooking assistant. "
    "You specialize in generating, adapting, and evaluating recipes based on user dietary needs and available ingredients. "
    "Always consider dietary profiles such as Hashimoto, diabetes, vegan, etc. "
    "Use the available tools to search, adapt, and explain recipes. Think step-by-step and provide clear, friendly responses."
    )

In [25]:
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage


class AgentState(TypedDict):
    messages: List[HumanMessage]

def debug_agent_logic(*args, **kwargs):    # zmieniam na inny rodzaj agenta
    print(f"Debug: Agent logic invoked with args={args}, kwargs={kwargs}")
    return create_react_agent(*args, **kwargs)

agent_node = debug_agent_logic(
    model = llm, 
    tools = tools,
    prompt = system_prompt,
    checkpointer=memory
)

builder = StateGraph(state_schema=AgentState)
builder.add_node("agent", agent_node)
builder.set_entry_point("agent")
builder.add_edge("agent", END)

graph = builder.compile()
graph = graph.with_config({"max_iterations": 3})  # NEW

Debug: Agent logic invoked with args=(), kwargs={'model': ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.3, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7a776da853f0>, default_metadata=()), 'tools': [Tool(name='SearchRecipeByIngredientsAndDiet', description="Searches for recipes based on given ingredients and an optional dietary profile. Input format: 'ingredients: zucchini, tofu | diet: Hashimoto'", func=<function debug_tool_wrapper.<locals>.wrapped_tool at 0x7a776d809360>), Tool(name='AdaptRecipeToDiet', description="Rewrites a recipe to make it compatible with a specific diet. Input format: 'title: [recipe title] | ingredients: [ingredients] | directions: [directions] | diet: [diet name]'", func=<function debug_tool_wrapper.<locals>.wrapped_tool at 0x7a776d8093f0>), Tool(name='GenerateMealPlan', description="Generates a 1-day healthy meal plan for a gi

In [26]:
# Check for structure in a text form
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	agent(agent)
	__end__([<p>__end__</p>]):::last
	__start__ --> agent;
	agent --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



### 4.3 Example interactions with the agent

In [27]:
# RAG check - does model search for recipes in the database?
response = graph.invoke({"messages": [HumanMessage(content="ingredients: zucchini, tofu | diet: Hashimoto")]} )

Debug: Invoking tool search_recipes_tool_wrapper with args=('ingredients: zucchini, tofu | diet: Hashimoto',), kwargs={}
Tool search_recipes_tool_wrapper 


100%|██████████| 1/1 [00:00<00:00,  3.21it/s]


Debug: Tool search_recipes_tool_wrapper returned Sautéed Zucchini  (distance: 0.40)
Ingredients: sautéed zucchini . ingredients: 1 medium zucchini, 2 tablespoons toasted sesame oil, kosher salt, freshly ground black pepper, gochugaru (coarse korean red pepper powder)

Grilled Vegetable Salad with Tofu  (distance: 0.41)
Ingredients: grilled vegetable salad with tofu . ingredients: 1 zucchini, halved lengthwise, 1 ear corn, husked, 1 bunch asparagus (about 16 pencil-thin spears), ends trimmed, 1 4-ounces firm tofu, cut into 2 1/2-inch slices, 4 scallions, olive oil cooking spray, 1 cup mixed greens, 1 romaine heart, chopped, 1/4 avocado, cut into bite-size chunks, 1 vine-ripened tomato, cut into bite-size chunks, 1 teaspoon dijon mustard, 1 teaspoon fresh lemon juice, 1 teaspoon fresh lime juice, 1/2 teaspoon white wine vinegar, 1/2 cup extra-virgin olive oil

Grilled Zucchini  (distance: 0.43)
Ingredients: grilled zucchini . ingredients: 3 large zucchini cut into 1/2-inch-thick slices, 

In [28]:
# AI generation: adapting recipe
user_input = """Get the recipe for Grilled Vegetable Salad with Tofu. Ingredients: grilled vegetable salad with tofu . ingredients: 1 zucchini, 
halved lengthwise, 1 ear corn, husked, 1 bunch asparagus (about 16 pencil-thin spears), ends trimmed, 1 4-ounces firm tofu, cut into 
2 1/2-inch slices, 4 scallions, olive oil cooking spray, 1 cup mixed greens, 1 romaine heart, chopped, 1/4 avocado, cut into bite-size chunks, 
1 vine-ripened tomato, cut into bite-size chunks, 1 teaspoon dijon mustard, 1 teaspoon fresh lemon juice, 1 teaspoon fresh lime juice, 
1/2 teaspoon white wine vinegar, 1/2 cup extra-virgin olive oil
Please adapt it to a High-Protein diet with chicken."""

response = graph.invoke(
    {"messages": [HumanMessage(content=user_input)]})

Debug: Invoking tool <lambda> with args=('title: Grilled Vegetable Salad with Tofu | ingredients: 1 zucchini, halved lengthwise, 1 ear corn, husked, 1 bunch asparagus (about 16 pencil-thin spears), ends trimmed, 1 4-ounces firm tofu, cut into 2 1/2-inch slices, 4 scallions, olive oil cooking spray, 1 cup mixed greens, 1 romaine heart, chopped, 1/4 avocado, cut into bite-size chunks, 1 vine-ripened tomato, cut into bite-size chunks, 1 teaspoon dijon mustard, 1 teaspoon fresh lemon juice, 1 teaspoon fresh lime juice, 1/2 teaspoon white wine vinegar, 1/2 cup extra-virgin olive oil | directions: unknown | diet: High-Protein',), kwargs={}
Tool adapt_recipe_to_diet_wrapper 
Tool adapt_recipe_to_diet 
Debug: Tool <lambda> returned Okay, here's a revised recipe for a High-Protein Grilled Vegetable Salad, focusing on boosting the protein content while keeping it delicious and healthy:

**Title: High-Protein Grilled Vegetable and Tofu Salad with Lemon-Herb Dressing**

**Focus:** This recipe incr

In [29]:
# generating meal plan for a whole day:
user_input = "diet: High-Protein"

response = graph.invoke(
    {"messages": [HumanMessage(content=user_input)]}
)

Debug: Invoking tool <lambda> with args=('diet: High-Protein',), kwargs={}
Tool generate_meal_plan_wrapper 
Tool generate_meal_plan 
Debug: Tool <lambda> returned Okay, here is a one-day high-protein meal plan, designed to be relatively simple and easy to prepare.  Remember to adjust portion sizes based on your individual caloric needs and activity level.

**Meal Type: Breakfast - Protein Power Oatmeal** - Oatmeal prepared with milk (dairy or unsweetened almond milk), mixed with a scoop of protein powder (whey, casein, or plant-based), topped with a handful of berries and a sprinkle of chopped nuts.

**Meal Type: Snack - Greek Yogurt with Almonds** - Plain Greek yogurt (choose a high-protein option) topped with a serving of almonds.

**Meal Type: Lunch - Grilled Chicken Salad** - Grilled chicken breast sliced over a bed of mixed greens, with chopped vegetables (cucumber, bell peppers, tomatoes), and a light vinaigrette dressing.

**Meal Type: Snack - Hard-Boiled Eggs** - Two hard-boile

In [30]:
# testing diet_explanation_tool
user_input = """title: Grilled Vegetable Salad with Tofu | 
ingredients: zucchini, corn, asparagus, tofu, scallions, greens, romaine, avocado, tomato, mustard, lemon juice, lime juice, vinegar, olive oil | 
diet: Hashimoto"""

response = graph.invoke(
    {"messages": [HumanMessage(content=user_input)]}
)

Debug: Invoking tool <lambda> with args=('title: Grilled Vegetable Salad with Tofu | ingredients: zucchini, corn, asparagus, tofu, scallions, greens, romaine, avocado, tomato, mustard, lemon juice, lime juice, vinegar, olive oil | diet: Hashimoto',), kwargs={}
Debug: Tool <lambda> returned This recipe is **mostly compatible** with a Hashimoto's diet, but with a few potential modifications depending on individual sensitivities. Here's the breakdown:

*   **Good:**

    *   **Most Vegetables (zucchini, asparagus, romaine, avocado, tomato, scallions, greens):** These are generally considered safe and beneficial for Hashimoto's, providing vitamins, minerals, and fiber.
    *   **Tofu:** Tofu can be problematic for some with Hashimoto's due to the soy content. However, it's generally considered fine for those who don't have soy sensitivities.
    *   **Olive Oil:** A healthy fat source and anti-inflammatory.
    *   **Lemon and Lime Juice:** Safe and beneficial.
    *   **Mustard:** As long

## 💡 5. Conversational Reasoning and Personalization

This section demonstrates how the AI Meal Coach agent engages in multi-turn conversation with memory and reasoning. We extend the agent's internal graph to support iterative reasoning and dynamic tool usage by adding routing and tool-execution logic.

### 5.1 LangGraph Agent: Reasoning Loop and Tool Handling

In [31]:
from langchain_core.messages import BaseMessage, ToolCall, ToolMessage

# change of agent state, so it tracks tool_calls and messages
class AgentState(TypedDict):
    messages: List[BaseMessage]
    tool_calls: Optional[List[ToolCall]]

